# 📄 이미지 PDF Textract OCR + 레이아웃(표) 추출 파이프라인

목적: 이미지 기반 PDF를 페이지 이미지로 변환한 뒤 AWS Textract로 OCR을 수행하고, 필요 시 표 구조를 분석하여 결과물을 저장하는 파이프라인입니다.

핵심 기능:
- .env에서 AWS 자격 증명/리전을 로드하여 boto3 클라이언트 생성
- PDF→페이지 이미지 변환(pdf2image), Textract DetectDocumentText로 페이지별 텍스트 추출
- 선택적으로 Textract AnalyzeDocument(TABLES)로 표 추출 및 텍스트 저장
- 전체 텍스트 파일, 페이지별 텍스트/이미지, 표 텍스트, 메타데이터(JSON) 저장

In [1]:
# ✅ 의존성 점검
import importlib

def check_lib(name):
    try:
        importlib.import_module(name)
        print(f'✔ {name} 사용 가능', flush=True)
        return True
    except Exception as e:
        print(f'✖ {name} 불가: {e}', flush=True)
        return False

needed = ['boto3', 'pdf2image', 'PIL', 'dotenv']
print('🔎 라이브러리 상태 확인:', flush=True)
status = {n: check_lib(n) for n in needed}
missing = [n for n, ok in status.items() if not ok]
if missing:
    print(f'⚠ 설치 필요: {", ".join(missing)}', flush=True)
else:
    print('✅ 모든 필수 라이브러리 확인됨', flush=True)

🔎 라이브러리 상태 확인:
✔ boto3 사용 가능
✔ boto3 사용 가능
✔ pdf2image 사용 가능
✔ PIL 사용 가능
✔ dotenv 사용 가능
✅ 모든 필수 라이브러리 확인됨
✔ pdf2image 사용 가능
✔ PIL 사용 가능
✔ dotenv 사용 가능
✅ 모든 필수 라이브러리 확인됨


In [2]:
# ⚙️ 경로/설정 및 AWS 초기화 (.env 로드)
import os, io, json, time
from pathlib import Path
from dataclasses import dataclass, field
from typing import Optional, Dict, Any, List
from dotenv import dotenv_values
import boto3

BASE_DIR = Path('/home/admin/wkms-aws/jupyter_notebook')
INPUT_PDF = BASE_DIR / 'data/input_docs/test.pdf'  # 필요시 변경
OUTPUT_DIR = BASE_DIR / 'data/textract_output/extracted_texts'
ENV_PATH = Path('/home/admin/wkms-aws/backend/.env')

@dataclass
class TextractConfig:
    aws_region: str = 'ap-northeast-2'
    ocr_dpi: int = 220
    use_tables: bool = True  # TABLES 분석 시도
    # 레이아웃/컬럼 처리 관련 설정
    column_mode: str = 'auto'        # 'auto' | 'single' | 'two' | 'n'
    column_split_min_gap: float = 0.08  # 좌우 컬럼 분할 최소 갭(0~1)
    max_columns: int = 3             # auto/n 모드에서 최대 컬럼 수 탐색
    min_lines_per_column: int = 3    # 너무 작은 컬럼은 무시하거나 병합

@dataclass
class Paths:
    output_root: Path = OUTPUT_DIR
    page_texts: Path = field(default_factory=lambda: OUTPUT_DIR)
    page_images: Path = field(default_factory=lambda: OUTPUT_DIR / 'page_images')
    page_tables: Path = field(default_factory=lambda: OUTPUT_DIR / 'page_tables')

# 출력 경로 준비
paths = Paths()
for d in [paths.output_root, paths.page_images, paths.page_tables]:
    d.mkdir(parents=True, exist_ok=True)

# .env 로드 및 AWS 세션/클라이언트 구성
env = dotenv_values(str(ENV_PATH)) if ENV_PATH.exists() else {}
aws_access_key_id = env.get('AWS_ACCESS_KEY_ID') or os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = env.get('AWS_SECRET_ACCESS_KEY') or os.environ.get('AWS_SECRET_ACCESS_KEY')
aws_session_token = env.get('AWS_SESSION_TOKEN') or os.environ.get('AWS_SESSION_TOKEN')
aws_region = env.get('AWS_REGION') or os.environ.get('AWS_REGION') or 'ap-northeast-2'

tex_cfg = TextractConfig(aws_region=aws_region)

if aws_access_key_id and aws_secret_access_key:
    session = boto3.Session(
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
        aws_session_token=aws_session_token,
        region_name=aws_region
    )
else:
    # 환경변수/프로파일/IMDS를 통해 자격증명 자동 탐색
    session = boto3.Session(region_name=aws_region)

textract = session.client('textract', region_name=aws_region)

print('📂 출력 경로 설정 완료:', flush=True)
print(f' - 페이지 텍스트: {paths.page_texts}', flush=True)
print(f' - 페이지 이미지: {paths.page_images}', flush=True)
print(f' - 표 출력: {paths.page_tables}', flush=True)
print('🔐 AWS 설정:', flush=True)
print(f' - REGION: {aws_region}', flush=True)
print(f' - ACCESS_KEY 제공 여부: {bool(aws_access_key_id)}', flush=True)
print('🧭 컬럼 설정:', flush=True)
print(f' - column_mode={tex_cfg.column_mode}, min_gap={tex_cfg.column_split_min_gap}, max_columns={tex_cfg.max_columns}, min_lines/col={tex_cfg.min_lines_per_column}', flush=True)

📂 출력 경로 설정 완료:
 - 페이지 텍스트: /home/admin/wkms-aws/jupyter_notebook/data/textract_output/extracted_texts
 - 페이지 이미지: /home/admin/wkms-aws/jupyter_notebook/data/textract_output/extracted_texts/page_images
 - 표 출력: /home/admin/wkms-aws/jupyter_notebook/data/textract_output/extracted_texts/page_tables
🔐 AWS 설정:
 - REGION: ap-northeast-2
 - ACCESS_KEY 제공 여부: True
🧭 컬럼 설정:
 - column_mode=auto, min_gap=0.08, max_columns=3, min_lines/col=3
 - 페이지 텍스트: /home/admin/wkms-aws/jupyter_notebook/data/textract_output/extracted_texts
 - 페이지 이미지: /home/admin/wkms-aws/jupyter_notebook/data/textract_output/extracted_texts/page_images
 - 표 출력: /home/admin/wkms-aws/jupyter_notebook/data/textract_output/extracted_texts/page_tables
🔐 AWS 설정:
 - REGION: ap-northeast-2
 - ACCESS_KEY 제공 여부: True
🧭 컬럼 설정:
 - column_mode=auto, min_gap=0.08, max_columns=3, min_lines/col=3


In [3]:
# 🔧 유틸리티
from typing import Tuple

def write_text(path: Path, content: str):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, 'w', encoding='utf-8') as f:
        f.write(content)

def write_json(path: Path, data: Dict[str, Any]):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

def is_image_pdf(pdf_path: Path, sample_pages: int = 3) -> bool:
    """간단 기준: pdfplumber로 앞부분 텍스트 추출 시 매우 적으면 이미지 기반으로 판단"""
    try:
        import pdfplumber  # type: ignore
        total_txt = ''
        with pdfplumber.open(str(pdf_path)) as pdf:
            for i, page in enumerate(pdf.pages[:sample_pages], 1):
                try:
                    total_txt += (page.extract_text() or '')
                except Exception:
                    pass
        return len((total_txt or '').strip()) < 100
    except Exception:
        # 라이브러리 없거나 실패 시 보수적으로 OCR 대상으로 간주
        return True

In [4]:
# 🔤 Textract OCR: PDF→이미지 변환 후 DetectDocumentText로 페이지별 텍스트 추출 (N컬럼 자동 정렬 포함)

def _kmeans_1d(values: List[float], k: int, iters: int = 15) -> List[int]:
    """아주 간단한 1D k-means 구현: values -> cluster indices. 안정성을 위해 중앙값 기반 초기화."""
    if k <= 1 or len(values) <= k:
        return list(range(len(values)))
    import math, statistics
    sorted_vals = sorted(set(values))
    if len(sorted_vals) < k:
        k = len(sorted_vals)
    # 초기 중심: quantile 분할
    centers = [sorted_vals[max(0, min(len(sorted_vals)-1, round((i+0.5)*len(sorted_vals)/k)-1))] for i in range(k)]
    assign = [0]*len(values)
    for _ in range(iters):
        # 할당
        for i, v in enumerate(values):
            ci = min(range(k), key=lambda j: abs(v - centers[j]))
            assign[i] = ci
        # 중심 업데이트
        new_centers = centers[:]
        for j in range(k):
            group = [values[i] for i, a in enumerate(assign) if a == j]
            if group:
                new_centers[j] = sum(group) / len(group)
        if all(abs(new_centers[j]-centers[j]) < 1e-4 for j in range(k)):
            break
        centers = new_centers
    return assign


def _split_into_n_columns(lines: List[Dict[str, Any]], max_cols: int, min_lines_per_col: int) -> List[List[Dict[str, Any]]]:
    if not lines:
        return [lines]
    xs = [l.get('Geometry', {}).get('BoundingBox', {}).get('Left', 0.0) for l in lines]
    cols = 1
    best_groups: List[List[Dict[str, Any]]] = [lines]
    for k in range(2, max_cols+1):
        assign = _kmeans_1d(xs, k)
        groups: List[List[Dict[str, Any]]] = [[] for _ in range(k)]
        for l, a in zip(lines, assign):
            groups[a].append(l)
        groups = [g for g in groups if len(g) >= min_lines_per_col]
        if len(groups) <= 1:
            continue
        # 컬럼 중심(Left 평균)으로 좌->우 정렬, 각 그룹은 위->아래->좌->우 정렬
        def left_mean(g):
            return sum([x.get('Geometry', {}).get('BoundingBox', {}).get('Left', 0.0) for x in g]) / len(g)
        groups.sort(key=left_mean)
        for g in groups:
            g.sort(key=lambda b: (b.get('Geometry', {}).get('BoundingBox', {}).get('Top', 0.0),
                                  b.get('Geometry', {}).get('BoundingBox', {}).get('Left', 0.0)))
        best_groups = groups
        cols = len(groups)
    return best_groups


def textract_detect_text_on_images(pdf_path: Path, paths: Paths, cfg: TextractConfig, textract_client) -> Dict[str, Any]:
    from pdf2image import convert_from_path  # type: ignore
    from PIL import Image  # type: ignore
    start = time.time()
    print(f'🔄 PDF→이미지 변환 중 (DPI={cfg.ocr_dpi})...', flush=True)
    images = convert_from_path(str(pdf_path), dpi=cfg.ocr_dpi)
    print(f'   ✅ 변환 완료: {len(images)}페이지', flush=True)

    full_text_parts: List[str] = []
    page_text_files: List[str] = []
    page_image_files: List[str] = []
    page_meta: List[Dict[str, Any]] = []

    for idx, img in enumerate(images, 1):
        p_start = time.time()
        print(f'   🔍 페이지 {idx}/{len(images)} Textract DetectDocumentText...', flush=True)
        # 이미지 저장 및 바이트 준비
        page_img_path = paths.page_images / f'{pdf_path.stem}_page_{idx:03d}.png'
        try:
            img.save(page_img_path)
            page_image_files.append(str(page_img_path))
        except Exception:
            pass
        buf = io.BytesIO()
        img.save(buf, format='PNG')
        img_bytes = buf.getvalue()

        resp = textract_client.detect_document_text(Document={'Bytes': img_bytes})
        blocks = resp.get('Blocks', [])
        lines = [b for b in blocks if b.get('BlockType') == 'LINE' and 'Text' in b]

        # 레이아웃 정렬
        if cfg.column_mode == 'single':
            groups = [lines]
        elif cfg.column_mode == 'two':
            groups = _split_into_n_columns(lines, max_cols=2, min_lines_per_col=cfg.min_lines_per_column)
        elif cfg.column_mode == 'n':
            groups = _split_into_n_columns(lines, max_cols=cfg.max_columns, min_lines_per_col=cfg.min_lines_per_column)
        else:  # 'auto'
            # 2열 우선 시도 → 충분치 않으면 n열 시도 → 실패 시 단일
            groups = _split_into_n_columns(lines, max_cols=2, min_lines_per_col=cfg.min_lines_per_column)
            if len(groups) <= 1 and cfg.max_columns > 2:
                groups = _split_into_n_columns(lines, max_cols=cfg.max_columns, min_lines_per_col=cfg.min_lines_per_column)
            if len(groups) <= 1:
                groups = [lines]
        # 그룹 내 정렬 보강
        for g in groups:
            g.sort(key=lambda b: (b.get('Geometry', {}).get('BoundingBox', {}).get('Top', 0.0),
                                  b.get('Geometry', {}).get('BoundingBox', {}).get('Left', 0.0)))
        # 좌→우 컬럼 순으로 병합
        ordered_text = ['\n'.join([b['Text'].strip() for b in g if b.get('Text')]).strip() for g in groups if g]
        txt = '\n\n'.join([t for t in ordered_text if t]).strip()

        page_txt_path = paths.page_texts / f'{pdf_path.stem}_page_{idx:03d}.txt'
        write_text(page_txt_path, txt)
        page_text_files.append(str(page_txt_path))
        full_text_parts.append(f'\n\n=== 페이지 {idx} ===\n' + txt)
        page_meta.append({
            'page': idx,
            'text_length': len(txt),
            'processing_time': round(time.time() - p_start, 3),
            'text_file': str(page_txt_path),
            'image_file': str(page_img_path) if page_image_files else None,
            'columns_detected': len(groups)
        })

    full_text = '\n'.join(full_text_parts).strip()
    full_txt_path = paths.output_root / f'{pdf_path.stem}_textract_full.txt'
    write_text(full_txt_path, full_text)

    meta = {
        'method': 'textract-detect',
        'pages': len(images),
        'aws_region': cfg.aws_region,
        'total_time': round(time.time() - start, 3),
        'page_details': page_meta,
        'output_file': str(full_txt_path),
        'page_text_files': page_text_files,
        'page_image_files': page_image_files
    }
    return {'text': full_text, 'metadata': meta}

In [5]:
# 📊 Textract TABLES: 저장된 페이지 이미지에 대해 AnalyzeDocument(TABLES)로 표 텍스트 저장

def textract_analyze_tables_on_images(pdf_path: Path, paths: Paths, textract_client, max_pages: Optional[int] = None) -> Dict[str, Any]:
    import base64
    saved: List[str] = []
    tables_summary: List[Dict[str, Any]] = []

    # 저장된 페이지 이미지 파일 목록을 순회
    page_imgs = sorted(paths.page_images.glob(f'{pdf_path.stem}_page_*.png'))
    if max_pages is not None:
        page_imgs = page_imgs[:max_pages]

    # 헬퍼: Block id 맵 및 텍스트 복원
    def build_block_map(blocks: List[Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
        return {b['Id']: b for b in blocks if 'Id' in b}

    def get_text_for_block(block_id: str, block_map: Dict[str, Dict[str, Any]]) -> str:
        block = block_map.get(block_id, {})
        if not block:
            return ''
        if block.get('BlockType') in ('WORD', 'SELECTION_ELEMENT') and 'Text' in block:
            return block.get('Text', '')
        text_parts: List[str] = []
        for rel in block.get('Relationships', []) or []:
            if rel.get('Type') == 'CHILD':
                for cid in rel.get('Ids', []) or []:
                    t = get_text_for_block(cid, block_map)
                    if t:
                        text_parts.append(t)
        return ' '.join(text_parts).strip()

    for page_idx, img_path in enumerate(page_imgs, 1):
        try:
            with open(img_path, 'rb') as f:
                img_bytes = f.read()
            print(f'   📑 표 분석(AnalyzeDocument-TABLES) 페이지 {page_idx}/{len(page_imgs)}...', flush=True)
            resp = textract_client.analyze_document(Document={'Bytes': img_bytes}, FeatureTypes=['TABLES'])
            blocks = resp.get('Blocks', [])
            block_map = build_block_map(blocks)
            # 페이지별 TABLE 블록만 선별
            page_tables = [b for b in blocks if b.get('BlockType') == 'TABLE' and b.get('Page') == page_idx]
            if not page_tables:
                continue
            for t_i, tbl in enumerate(page_tables, 1):
                # CELL 블록 수집 후 행/열 인덱스로 정렬
                cells = [b for b in blocks if b.get('BlockType') == 'CELL' and b.get('Page') == page_idx]
                if not cells:
                    continue
                max_row = max([c.get('RowIndex', 0) for c in cells]) if cells else 0
                max_col = max([c.get('ColumnIndex', 0) for c in cells]) if cells else 0
                grid: Dict[int, Dict[int, str]] = {r: {} for r in range(1, max_row + 1)}
                for cell in cells:
                    r = cell.get('RowIndex', 0)
                    c = cell.get('ColumnIndex', 0)
                    txt = get_text_for_block(cell.get('Id', ''), block_map)
                    if r and c:
                        grid.setdefault(r, {})[c] = (txt or '').strip()
                lines: List[str] = []
                for r in range(1, max_row + 1):
                    row_vals = [grid.get(r, {}).get(c, '') for c in range(1, max_col + 1)]
                    lines.append(' | '.join([v.strip() for v in row_vals]))
                content = (f'=== 페이지 {page_idx} 테이블 {t_i} ===\n' + '\n'.join(lines)).strip()
                out = paths.page_tables / f'{pdf_path.stem}_p{page_idx:03d}_t{t_i:02d}.txt'
                write_text(out, content)
                saved.append(str(out))
                tables_summary.append({'page': page_idx, 'table_index': t_i, 'rows': len(lines), 'cols': max_col})
        except Exception as e:
            print(f'⚠ 표 분석 실패(page {page_idx}): {e}', flush=True)
            continue
    return {'tables': tables_summary, 'files': saved}

In [6]:
# 🧠 오케스트레이터 (컬럼 모드 전달 포함)

def process_pdf_with_textract(pdf_path: Path, paths: Paths, cfg: TextractConfig, textract_client) -> Dict[str, Any]:
    print(f'📥 입력 파일: {pdf_path}', flush=True)
    if not pdf_path.exists():
        raise FileNotFoundError(f'파일이 없습니다: {pdf_path}')
    img_pdf = is_image_pdf(pdf_path)
    print(f'🧪 이미지 PDF 판별: {"예" if img_pdf else "아니오"}', flush=True)
    print(f'🧭 컬럼 모드: {cfg.column_mode}, min_gap={cfg.column_split_min_gap}', flush=True)

    # 1) Textract OCR
    res = textract_detect_text_on_images(pdf_path, paths, cfg, textract_client)

    # 2) 표 추출(선택)
    table_info = {'tables': [], 'files': []}
    if cfg.use_tables:
        print('📊 표 추출 시도(Textract TABLES)', flush=True)
        table_info = textract_analyze_tables_on_images(pdf_path, paths, textract_client)

    # 3) 메타데이터 합성
    meta = res.get('metadata', {})
    meta.update({
        'tables': table_info.get('tables', []),
        'table_files': table_info.get('files', [])
    })
    doc_meta_path = paths.output_root / f'{pdf_path.stem}_textract_metadata.json'
    write_json(doc_meta_path, meta)

    print('✅ 처리 완료', flush=True)
    print(f' - 전체 텍스트 파일: {meta.get("output_file")}', flush=True)
    print(f' - 표 파일 수: {len(meta.get("table_files", []))}', flush=True)
    print(f' - 페이지 텍스트 파일 수: {len(meta.get("page_text_files", []))}', flush=True)
    print(f' - 페이지 이미지 파일 수: {len(meta.get("page_image_files", []))}', flush=True)
    return {'text': res.get('text', ''), 'metadata': meta}

In [7]:
# ▶️ 실행 러너: test.pdf
pdf_path = INPUT_PDF
print('🚀 Textract OCR + 표 추출 데모 시작', flush=True)
print(f'입력: {pdf_path}', flush=True)
result = process_pdf_with_textract(pdf_path, paths, tex_cfg, textract)

# 상위 500자 미리보기
preview = (result.get('text') or '')[:500]
print('\n🔎 전체 텍스트 미리보기(상위 500자):', flush=True)
print(preview if preview else '(미리보기 없음)', flush=True)

🚀 Textract OCR + 표 추출 데모 시작
입력: /home/admin/wkms-aws/jupyter_notebook/data/input_docs/test.pdf
📥 입력 파일: /home/admin/wkms-aws/jupyter_notebook/data/input_docs/test.pdf
🧪 이미지 PDF 판별: 예
🧭 컬럼 모드: auto, min_gap=0.08
🔄 PDF→이미지 변환 중 (DPI=220)...
입력: /home/admin/wkms-aws/jupyter_notebook/data/input_docs/test.pdf
📥 입력 파일: /home/admin/wkms-aws/jupyter_notebook/data/input_docs/test.pdf
🧪 이미지 PDF 판별: 예
🧭 컬럼 모드: auto, min_gap=0.08
🔄 PDF→이미지 변환 중 (DPI=220)...
   ✅ 변환 완료: 27페이지
   🔍 페이지 1/27 Textract DetectDocumentText...
   ✅ 변환 완료: 27페이지
   🔍 페이지 1/27 Textract DetectDocumentText...
   🔍 페이지 2/27 Textract DetectDocumentText...
   🔍 페이지 2/27 Textract DetectDocumentText...
   🔍 페이지 3/27 Textract DetectDocumentText...
   🔍 페이지 3/27 Textract DetectDocumentText...
   🔍 페이지 4/27 Textract DetectDocumentText...
   🔍 페이지 4/27 Textract DetectDocumentText...
   🔍 페이지 5/27 Textract DetectDocumentText...
   🔍 페이지 5/27 Textract DetectDocumentText...
   🔍 페이지 6/27 Textract DetectDocumentText...
   🔍 페이지 6/27 Textr

## 참고
- /home/admin/wkms-aws/backend/.env 파일에서 AWS_REGION, AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY(필요시 AWS_SESSION_TOKEN) 등을 읽어 사용합니다.
- Textract DetectDocumentText는 이미지(예: PNG/JPG) 바이트에 동기식으로 사용할 수 있습니다. 본 노트북은 PDF를 페이지 이미지로 변환 후 사용합니다.
- 표 추출은 AnalyzeDocument(TABLES)를 이미지에 대해 동기식으로 호출하여 기본 텍스트 형태로 저장합니다(간단한 복원).
- 비용/호출 한도에 유의하세요. 대용량 문서는 처리 시간이 길어질 수 있습니다.
- 출력물:
  - 전체 텍스트 파일: <파일명>_textract_full.txt
  - 페이지별 텍스트 파일: <파일명>_page_###.txt
  - 페이지 이미지: page_images/ 폴더 내 PNG
  - 표 텍스트: page_tables/ 폴더 내 텍스트 파일
  - 문서 메타데이터 JSON: <파일명>_textract_metadata.json